Import the necessary **modules**.

In [3]:
from tree_sitter import Language, Parser

Create separate languages.

In [4]:
JA_LANGUAGE = Language('../../tree-sitter/build/my-languages.so', 'java')
C_LANGUAGE = Language('../../tree-sitter/build/my-languages.so', 'c')
CS_LANGUAGE = Language('../../tree-sitter/build/my-languages.so', 'c_sharp')
CPP_LANGUAGE = Language('../../tree-sitter/build/my-languages.so', 'cpp')

Create separate **parsers**.

In [5]:
parser_JA = Parser()
parser_JA.set_language(JA_LANGUAGE)

parser_C = Parser()
parser_C.set_language(C_LANGUAGE)

parser_CS = Parser()
parser_CS.set_language(CS_LANGUAGE)

parser_CPP = Parser()
parser_CPP.set_language(CPP_LANGUAGE)

Write a function to parse the necessary **files**, gather the blocks of necessary types.

In [6]:
def ParseFile(file, language):
    with open(file, 'r') as fin:
        code = bytes(fin.read(), 'utf-8')
    if language == 'Java':
        tree = parser_JA.parse(code)
        good_types = ['constructor_declaration', 'method_declaration']
    elif language == 'C':
        tree = parser_C.parse(code)
        good_types = ['function_definition']
    elif language == 'CS':
        tree = parser_CS.parse(code)
        good_types = ['method_declaration', 'indexer_declaration', 'property_declaration']
    elif language == 'CPP':
        tree = parser_CPP.parse(code)
        good_types = ['function_definition']
    root = tree.root_node
    functions = []
    def addChildrenNodes(Node):
        for child in Node.children:
            if child.type in good_types:
                functions.append(child)
            if len(child.children) != 0:
                addChildrenNodes(child)
    addChildrenNodes(root)
    for function in functions:
        print('>>>', function.type, '\n\n', code[function.start_byte:function.end_byte].decode('utf-8'), '\n')

Parse the file.

In [7]:
ParseFile('examples/WinLauncher.cpp', 'CPP')

>>> function_definition 

 std::string EncodeWideACP(const std::wstring &str)
{
  int cbANSI = WideCharToMultiByte(CP_ACP, 0, str.c_str(), str.size(), NULL, 0, NULL, NULL);
  char* ansiBuf = new char[cbANSI];
  WideCharToMultiByte(CP_ACP, 0, str.c_str(), str.size(), ansiBuf, cbANSI, NULL, NULL);
  std::string result(ansiBuf, cbANSI);
  delete[] ansiBuf;
  return result;
} 

>>> function_definition 

 std::string LoadStdString(int id)
{
  wchar_t *buf = NULL;
  int len = LoadStringW(hInst, id, reinterpret_cast<LPWSTR>(&buf), 0);
  return len ? EncodeWideACP(std::wstring(buf, len)) : "";
} 

>>> function_definition 

 bool FileExists(const std::string& path)
{
  return GetFileAttributesA(path.c_str()) != INVALID_FILE_ATTRIBUTES;
} 

>>> function_definition 

 bool IsValidJRE(const char* path)
{
  std::string dllPath(path);
  if (dllPath[dllPath.size() - 1] != '\\')
  {
    dllPath += "\\";
  }
  return FileExists(dllPath + "bin\\server\\jvm.dll") || FileExists(dllPath + "bin\\client\\jvm